# Filtro de spam


#### [Entrenamiento:](#Entrenamiento)
- def que separe una línea en palabras y las agregue a un conjunto dado por parámetro
- def de un generator que devuelva un set de palabras por cada correo de un mbox dado por parámetro
- def que devuelve Dict de la forma {'w' : n } con w = palabra y n = Hw para 'ham.mbox' o n = Sw para 'spam.box'

#### [Clasificación:](#Clasificación)
- def que devuelva P(y|xw) para cada palabra w de un conjunto de palabras, uno por cada correo de 'news.box'
- def que devuelva lista de 15 palabras de mejor clasificación individual de un correo nuevo
- def que devuelva P(y|x1, . . . , x15) de la lista anterior
- def que devuelva lista de 0 o 1 por cada correo de 'news.mbox' en el mismo orden

#### [Incorporación:](#Incorporación)
- def que devuelva un set de palabras de un 'new-spam.mbox' o 'new-ham.mbox' que contiene un solo mensaje
- def que combine el anterior conjunto al dict de entrenamiento aumentando en 1 cada valor
- def que escriba el nuevo correo 'new-spam.mbox' o 'new-ham.mbox' en el fichero que corresponda

#### [Uso:](#Uso)
- Ejemplo de uso de los 3 procedimientos anteriores

# Entrenamiento

In [10]:
import re


# Separa al string 'line' según el regex delimiter
# Y añade los trozos al set 'words' pasado por parámetro
def process_line(line, words, delimiter = '[,. ()]|\n|\t'):
    for word in re.split(delimiter, line):
        words.add(word)


# Devuelve un set de palabras diferentes de un solo correo
def words_per_mail(mbox):    
    with open(mbox) as fp:
        lines = iter(fp.readlines())
        while True:
            words = set()
            for subject in lines:
                if subject.startswith("Subject: "):
                    break
            else:
                break

            process_line(subject[9:], words)

            for status in lines:
                if status == "X-KMail-MDN-Sent:  \n":
                    break

            intro = fromOne = False
            for line in lines:
                if line == '\n' :
                    intro = True
                    continue
                elif line.startswith('From: ') and fromOne :
                    break
                elif line.startswith('From ') and intro :
                    fromOne = True
                else:
                    intro = False
                    process_line(line, words)

            words.remove('')
            yield words


# Agrega words a dictionary, incrementando la ocurrencia en 1
def merge_to_dict(words, dictionary):
    for w in words:
        dictionary[w] = dictionary[w] + 1 if w in dictionary else 1


# Devuelve dict de palabras no repetidas y el número de correos
# en los que aparecen, del mbox dado por parámetro
def training_dict(mbox):
    dictionary = {}
    n = 0
    for words in words_per_mail(mbox):
        merge_to_dict(words, dictionary)
        n += 1
    return dictionary, n



In [11]:
# Devuelve dict de la forma {'word' : Sw }
# y el número de correos para spambox
# y otro dict de la forma {'word' : Hw }
# y el número de correos para hambox
def training(spambox, hambox):
    spam_dict, S = training_dict(spambox)
    ham_dict, H = training_dict(hambox)
    return spam_dict, S, ham_dict, H


# Clasificación

In [12]:
# Devuelve generator que recorre las palabras de cada correo de newsbox
# Devuelve un set por cada llamada, que corresponde al siguiente correo
def news_words_generator(newsbox = './news.mbox'):
    return words_per_mail(newsbox)

#Devuelve P(y|xw) por cada palabra en un correo
def probPerWord(words):
    #S = número de correos spam
    #H = número de correos ham
    #spam_dict = dict de la forma {'word' : Sw }
    #ham_dict = dict de la forma {'word' : Hw }
    spam_dict, S, ham_dict, H = training('./spam.mbox', './ham.mbox')
    Py = S/(S + H)
    
    Pxwy = []
    Pw = []
    Pyxw = []
    Pxwnoy = []
    for word in words:
        if word in spam_dict.keys():
            Sw = spam_dict[word] #número de correos Spam donde aparece esa palabra
        else:
            Sw = 0
        Pxwy.append(Sw/S)
        if word in ham_dict.keys():
            Hw = ham_dict[word] # número de correos no Spam donde aparece esa palabra
        else:
            Hw = 0
        Pxwnoy.append(Hw/H)
        if Sw==0 or Hw==0:
            Pw.append(0.5)
        else:
            Pw.append((Sw+Hw)/(S+H))
    #Calculamos P(y|xw) con: P(xw|y)*P(y)/P(xw)
    counter2 = 0
    for value in Pxwy:
        Pyxw.append(value*Py/Pw[counter2])
        counter2+=1
    return Pyxw
                
                

In [45]:
from collections import namedtuple

Pair = namedtuple("Pair", ["value", "position"])

#Devuelve una lista de 15 palabras de mejor clasificación individual de un correo
def bestQualificationWords(words):
    probabilidades = probPerWord(words)
    wordsList = list(words)
    diffZeroOne = []
    bestFifteen = []
    result = []
    #Calculamos la diferencia entre las probabilidades y 0 ó 1 (dependiendo de su valor), para ver cuáles son
    #los más próximos a éstos y escoger los 15 mejores
    counter = 0
    for prob in probabilidades: 
        if prob <= 0.5:
            diffZeroOne.append(Pair(abs(prob), counter))
        elif prob > 0.5:
            diffZeroOne.append(Pair(abs(1-prob), counter))
        counter+=1
    #Ordenamos de menor a mayor
    diffSorted = sorted(diffZeroOne)
    #Seleccionamos los 15 primeros (los de menor diferencia con 0 ó 1)
    i = 0
    while i<15:
        if (i < len(diffSorted)):
            bestFifteen.append(Pair(diffSorted[i].value, diffSorted[i].position))
            i+=1
    #Por último, devolvemos las palabras que se corresponden a esos resultados
    for best in bestFifteen:
        result.append(wordsList[best.position])
    #También devolvemos Pxwy y Pxwnoy de estas 15 palabras para usarlos después
    Pxwy = []
    Pxwnoy = []
    for w in result:
        if w in spam_dict.keys():
            Sw = spam_dict[w] #número de correos Spam donde aparece esa palabra
        else:
            Sw = 0
        Pxwy.append(Sw/S)
        if w in ham_dict.keys():
            Hw = ham_dict[w] # número de correos no Spam donde aparece esa palabra
        else:
            Hw = 0
        Pxwnoy.append(Hw/H)
    print(Pxwy, Pxwnoy)
    return Pxwy, Pxwnoy

In [37]:
#Devuelve P(y|x1,...,x15)
def probOfBeingSpam(Pxwy, Pxwnoy):
    spam_dict, S, ham_dict, H = training('./spam.mbox', './ham.mbox')
    Si = 1
    Hi = 1
    Py = S/(S + H)
    Pnoy = H/(S + H)
    var1 = 1
    for num in Pxwy:
        var1 = var1*Py*num
    var2 = 1
    for num2 in Pxwnoy:
        var2 = var2*Pnoy*num2
    Pyxn = var1/(var1 + var2)
    return Pyxn
    

In [40]:
# Función que recibe un mbox de correos nuevos
# y que devuelva una lista de 0 o 1 para cada correo nuevo 
def clasification(newsbox):
    wordsPerMail = words_per_mail(newsbox)
    clas = []
    for mailWords in wordsPerMail:
        Pxwy, Pxwnoy = bestQualificationWords(mailWords)
        probs = probOfBeingSpam(Pxwy, Pxwnoy)
        if probs > 0.9:
            clas.append(1)
            print("hola")
        else:
            clas.append(0)
            print("hola2")
    return clas
    

# Incorporación

In [16]:
# Escribe el nuevo correo ubicado en la ruta 'new'
# en el fichero de ruta 'box'
def append_new_to_mbox(new, mbox):
    with open(mbox, 'a') as writable:
        with open(new) as readable:
            writable.write(readable.read())


# Actualiza dictionary a partir del set de palabras
# del único mensaje contenido en 'new_box'
def update_dict(new_box, dictionary):
    new_words = next(words_per_mail(new_box))
    merge_to_dict(new_words, dictionary)


In [17]:
# Actualiza dictionary y escribe el contenido de new_box en mbox
def incorporation(dictionary, new_box, mbox):
    update_dict(new_box, dictionary)
    append_new_to_mbox(new_box, mbox)


# Uso

In [46]:
# Archivos
newspam = './new-spam.mbox'
newham = './new-ham.mbox'
spambox = './spam.mbox'
hambox = './ham.mbox'

# Entrenamiento
spam_dict, S, ham_dict, H = training(spambox, hambox)

# Clasificación
  #probPerWord
wordsPerMail = words_per_mail('./news.mbox')
print(probPerWord(next(wordsPerMail)))
  #bestQualificationWords
Pxwy, Pxwnoy = bestQualificationWords(next(wordsPerMail))
print(best, Pxwy, Pxwnoy)
  #probOfBeingSpam
print(probOfBeingSpam(Pxwy, Pxwnoy))  
  #clasification
cla =  clasification('./news.mbox')

# Incorporación a spam.box
incorporation(spam_dict, newspam, spambox)

# Incorporación a ham.box
incorporation(ham_dict, newham, hambox)

[0.0, 0.375, 0.0, 0.3333333333333333, 0.6, 0.5, 0.0, 0.6781609195402298, 0.0, 0.027586206896551724, 0.23076923076923078, 0.027586206896551724, 0.6666666666666666, 0.013793103448275862, 0.75]
[0.08333333333333333, 0.047619047619047616, 0.011904761904761904, 0.03571428571428571, 0.03571428571428571, 0.023809523809523808, 0.023809523809523808, 0.023809523809523808, 0.023809523809523808, 0.047619047619047616, 0.023809523809523808, 0.15476190476190477, 0.14285714285714285, 0.27380952380952384, 0.17857142857142858] [0.01639344262295082, 0.01639344262295082, 0.04918032786885246, 0.01639344262295082, 0.13114754098360656, 0.01639344262295082, 0.01639344262295082, 0.01639344262295082, 0.01639344262295082, 0.03278688524590164, 0.01639344262295082, 0.11475409836065574, 0.11475409836065574, 0.5737704918032787, 0.16393442622950818]
['$0', 'quick', 'performance', 'experts', 'last', 'effects', 'Sex', 'because:', 'erection', 'better', 'Need', 'take', 'than', 'at', 'no'] [0.08333333333333333, 0.04761904

ZeroDivisionError: float division by zero